In [1]:
#import library
from libsbml import readSBML
from cobra import io
from cobra.core import Metabolite

#import sbml file
model = io.sbml.create_cobra_model_from_sbml_file("/home/sanu/Documents/Scripts/git/plantcoremetabolism-model/PlantCoreMetabolism_v1_2_3.xml")

#change Biomass_tx to Protein_biomass
rxn = model.reactions.get_by_id("Biomass_tx")
rxn.id = "Protein_biomass_tx"
for met in ["K_b","Ca_b","Mg_b"]:
    met  = model.metabolites.get_by_id(met)
    coeff = rxn.metabolites.get(met)
    rxn.add_metabolites({met:-1*coeff})
met = Metabolite("Protein_b",name="Protein_biomass")
formula_dict = rxn.check_mass_balance()
met.formula = "".join([atom+str(formula_dict[atom]*-1) for atom in formula_dict.keys() if atom != "charge"])
met.charge = formula_dict["charge"]*-1
rxn.add_metabolites({met:1})

In [2]:
import pandas as pd
biomass = pd.DataFrame(data={"":["sSUCROSE_b","GLC_c","FRU_c","Starch_b","Cellulose_b","PALMITATE_c",
                                 "L_PHOSPHATIDATE_p","PHOSPHATIDYL_CHOLINE_r",
                                 "L_1_PHOSPHATIDYL_ETHANOLAMINE_r","DIACYLGLYCEROL_p","Protein_b","sMAL_b",
                                 "sCIT_b","sFUM_b","ARG_c","HIS_c","LYS_c","sASP_b",
                                 "sGLU_b","sSER_b","THR_c","ASN_c","sGLN_b","CYS_c",
                                 "GLY_c","PRO_c","sALA_b","VAL_c","ILE_c","LEU_c",
                                 "MET_c","PHE_c","TYR_c","TRP_c","sGABA_b"],
                             "leaf":[0.0]*35,"stem":[0.0]*35,"root":[0.0]*35,"seed":[0.0]*35,},dtype="float64")
biomass = biomass.set_index("")

### Leaf

In [3]:
proteinMW = model.metabolites.get_by_id("Protein_b").formula_weight

cobra/core/metabolite.py:103 UserWarning: 17.2020450391 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)
cobra/core/metabolite.py:103 UserWarning: 25.9492553613 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)
cobra/core/metabolite.py:103 UserWarning: 5.7497158698 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)
cobra/core/metabolite.py:103 UserWarning: 4.9536245179 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)
cobra/core/metabolite.py:103 UserWarning: 0.1420323181 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)


In [4]:
biomass.at["sSUCROSE_b","leaf"]=5.43252595155707/model.metabolites.get_by_id("SUCROSE_c").formula_weight
biomass.at["Starch_b","leaf"]=12.2894841499199/model.metabolites.get_by_id("STARCH_p").formula_weight
biomass.at["GLC_c","leaf"]=5.43252595155707/model.metabolites.get_by_id("GLC_c").formula_weight
biomass.at["FRU_c","leaf"]=12.2894841499199/model.metabolites.get_by_id("FRU_c").formula_weight
biomass.at["Cellulose_b","leaf"]=17.3121714956894/model.metabolites.get_by_id("STARCH_p").formula_weight
biomass.at["L_PHOSPHATIDATE_p","leaf"]=14.2253521126762
biomass.at["PHOSPHATIDYL_CHOLINE_r","leaf"]=4.71830985915491
biomass.at["sASP_b","leaf"]=0.050490196078431/model.metabolites.get_by_id("L_ASPARTATE_c").formula_weight
biomass.at["sSER_b","leaf"]=0.030894308943082/model.metabolites.get_by_id("SER_c").formula_weight
biomass.at["THR_c","leaf"]=0.293495934959346/model.metabolites.get_by_id("THR_c").formula_weight
biomass.at["ASN_c","leaf"]=1.20588235294117/model.metabolites.get_by_id("ASN_c").formula_weight
biomass.at["PHE_c","leaf"]=0.014318181818182/model.metabolites.get_by_id("PHE_c").formula_weight
biomass.at["TYR_c","leaf"]=0.034065934065932/model.metabolites.get_by_id("TYR_c").formula_weight

### Stem



In [5]:
biomass.at["sSUCROSE_b","stem"] =  3.65614798694232/model.metabolites.get_by_id("SUCROSE_c").formula_weight
biomass.at["GLC_c","stem"]=8.29162132752991/model.metabolites.get_by_id("GLC_c").formula_weight
biomass.at["FRU_c","stem"]=6.98585418933624/model.metabolites.get_by_id("FRU_c").formula_weight
biomass.at["Cellulose_b","stem"]=28.8030467899891/model.metabolites.get_by_id("STARCH_p").formula_weight
biomass.at["sMAL_b","stem"] = 1.52980613103922/model.metabolites.get_by_id("MAL_c").formula_weight
biomass.at["sASP_b","stem"]=0.340245506224343/model.metabolites.get_by_id("L_ASPARTATE_c").formula_weight
biomass.at["sSER_b","stem"]=0.323363095610159/model.metabolites.get_by_id("SER_c").formula_weight
biomass.at["sGLN_b","stem"]=0.055408937400402/model.metabolites.get_by_id("GLN_c").formula_weight
biomass.at["CYS_c","stem"]=0.077918818219315/model.metabolites.get_by_id("CYS_c").formula_weight
biomass.at["PRO_c","stem"]=0.130514020517353/model.metabolites.get_by_id("PRO_c").formula_weight
biomass.at["sALA_b","stem"]=1.11683639447685/model.metabolites.get_by_id("L_ALPHA_ALANINE_c").formula_weight
biomass.at["VAL_c","stem"]=1.22852003392454/model.metabolites.get_by_id("VAL_c").formula_weight
biomass.at["LEU_c","stem"]=5.70755343456485/model.metabolites.get_by_id("LEU_c").formula_weight

### Seed


In [6]:
biomass.at["sSUCROSE_b","seed"] = 0.414/model.metabolites.get_by_id("SUCROSE_c").formula_weight
biomass.at["GLC_c","seed"]=1.654/model.metabolites.get_by_id("GLC_c").formula_weight
biomass.at["FRU_c","seed"]=1.594/model.metabolites.get_by_id("FRU_c").formula_weight
biomass.at["L_PHOSPHATIDATE_p","leaf"]=14.2253521126762/model.metabolites.get_by_id("L_PHOSPHATIDATE_p").formula_weight
biomass.at["Protein_b","leaf"]=8.339/proteinMW
biomass.at["ARG_c","seed"] = 0.509/model.metabolites.get_by_id("ARG_c").formula_weight
biomass.at["HIS_c","seed"] = 0.287/model.metabolites.get_by_id("HIS_c").formula_weight
biomass.at["LYS_c","seed"] = 0.282/model.metabolites.get_by_id("LYS_c").formula_weight
biomass.at["sASP_b","seed"] = 7.283/model.metabolites.get_by_id("L_ASPARTATE_c").formula_weight
biomass.at["sGLU_b","seed"] = 21.617/model.metabolites.get_by_id("sGLU_b").formula_weight
biomass.at["sSER_b","seed"] = 0.613/model.metabolites.get_by_id("sSER_b").formula_weight
biomass.at["THR_c","seed"] = 0.426/model.metabolites.get_by_id("THR_c").formula_weight
biomass.at["CYS_c","seed"] = 0.115/model.metabolites.get_by_id("CYS_c").formula_weight
biomass.at["GLY_c","seed"] = 0.385/model.metabolites.get_by_id("GLY_c").formula_weight
biomass.at["PRO_c","seed"] = 1.129/model.metabolites.get_by_id("PRO_c").formula_weight
biomass.at["sALA_b","seed"] = 1.27/model.metabolites.get_by_id("L_ALPHA_ALANINE_c").formula_weight
biomass.at["VAL_c","seed"] = 0.713/model.metabolites.get_by_id("VAL_c").formula_weight
biomass.at["ILE_c","seed"] = 0.572/model.metabolites.get_by_id("ILE_c").formula_weight
biomass.at["LEU_c","seed"] = 1.861/model.metabolites.get_by_id("LEU_c").formula_weight
biomass.at["MET_c","seed"] = 0.207/model.metabolites.get_by_id("MET_c").formula_weight
biomass.at["PHE_c","seed"] = 0.73/model.metabolites.get_by_id("PHE_c").formula_weight
biomass.at["TYR_c","seed"] = 0.544/model.metabolites.get_by_id("TYR_c").formula_weight
biomass.at["TRP_c","seed"] = 0.136/model.metabolites.get_by_id("TRP_c").formula_weight
biomass.to_csv("/home/sanu/biomass_sorghum.csv")

### Mature leaves

Extract day-time starch:sucrose accumulation rate ratio from Lunn and Hatch 1995, Figure 1

<img src="references/Screenshot_20190714_104104.png" height=25% width=25%>

In [10]:
df_matureleaf_content_1 = pd.DataFrame(data={"Time":["Dawn","Dusk"],
                                             "Hexose (umol hexose/mgChl)":[24.126984126984112-22.222222222222204,
                                                                           121.26984126984127-116.19047619047619],
                                             "Sucrose (umol hexose/mgChl)":[22.222222222222204-17.142857142857146,
                                                                            116.19047619047619-95.87301587301586],
                                             "Starch (umol hexose/mgChl)":[17.142857142857146,95.87301587301586]})

In [11]:
Delta_sucrose = float(df_matureleaf_content_1["Sucrose (umol hexose/mgChl)"][df_matureleaf_content_1["Time"]=="Dusk"]) - \
float(df_matureleaf_content_1["Sucrose (umol hexose/mgChl)"][df_matureleaf_content_1["Time"]=="Dawn"])

Delta_hexose =float(df_matureleaf_content_1["Hexose (umol hexose/mgChl)"][df_matureleaf_content_1["Time"]=="Dusk"]) - \
float(df_matureleaf_content_1["Hexose (umol hexose/mgChl)"][df_matureleaf_content_1["Time"]=="Dawn"])

Delta_starch = float(df_matureleaf_content_1["Starch (umol hexose/mgChl)"][df_matureleaf_content_1["Time"]=="Dusk"]) - \
float(df_matureleaf_content_1["Starch (umol hexose/mgChl)"][df_matureleaf_content_1["Time"]=="Dawn"])

df_matureleaf_content_1["Starch (dusk-dawn)"]=["-",Delta_starch]
df_matureleaf_content_1["Sucrose (dusk-dawn)"]=["-",Delta_sucrose]
df_matureleaf_content_1["Hexose (dusk-dawn)"]=["-",Delta_hexose]
df_matureleaf_content_1

,Hexose (umol hexose/mgChl),Starch (umol hexose/mgChl),Sucrose (umol hexose/mgChl),Time,Starch (dusk-dawn),Sucrose (dusk-dawn),Hexose (dusk-dawn)
0,1.904762,17.142857,5.079365,Dawn,-,-,-
1,5.079365,95.873016,20.317460,Dusk,78.7302,15.2381,3.1746


In [12]:
Sucrose_in_umols = 0.5*(df_matureleaf_content_1.at[1,"Sucrose (dusk-dawn)"])
Starch_in_umols = (df_matureleaf_content_1.at[1,"Starch (dusk-dawn)"])
Starch_to_sucrose_ratio = Starch_in_umols/Sucrose_in_umols
print ("Starch sucrose day-time accumulation rate = "+str(Starch_to_sucrose_ratio))

Starch sucrose day-time accumulation rate = 10.3333333333
